# What do we have to do?

- Data acquisition
- Preparation
- Cleaning and aggregation
- Exploratory data analyis


# Description

xx
xx
xx

***

# Data
Main datasets:
- **movies_metadata.csv** - Movies dataset
- **credits_cleaned01.csv** - List of people per movie (directors, actors, etc)

***


## Movies

In [38]:
import pandas as pd
movies = pd.read_csv('data/movies_metadata.csv',dtype={'popularity': 'float'}, na_values=['Beware Of Frost Bites'])
print('Columns:')
print(',  '.join(movies))

Columns:
adult,  belongs_to_collection,  budget,  genres,  homepage,  id,  imdb_id,  original_language,  original_title,  overview,  popularity,  poster_path,  production_companies,  production_countries,  release_date,  revenue,  runtime,  spoken_languages,  status,  tagline,  title,  video,  vote_average,  vote_count


In [39]:
movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


***
## CREDITS
This dataframe contains 3 columns:
- **cast**: Json string with the information about the actor/acrtrees
- **crew**: Json string with the information about Directors and other members of the team
- **id**: Movie identifier

***

First we import json to to be able to deal with json strings:

In [434]:
import json
from os.path import exists

**credits_cleaned01** is big in size, the first action we are doing here is removing the column **crew** since we are not going to use it and then we save the dataset as a new file (which size is almost a half of the original dataset)

## REDUCING CREDITS FILE SIZE

**RUN ALL THIS IF YOU HAVE THE credits_cleaned01.csv FILE,**

This file is greater than 100mb, we can not upload it into github, skip this part and run the section where we import the file **credits_cleaned03.csv** down below.

In [477]:
file_exists = exists("data/credits_cleaned01.csv")
if(file_exists):
    credits = pd.read_csv('data/credits_cleaned01.csv')
    credits.drop(columns=['crew'], inplace=True, axis=1)
    credits.to_csv('data/credits_cleaned02.csv', index=False)
    display(list(credits.columns))

['Unnamed: 0', 'cast', 'id']

Now we read again the new file  and displays its size and columns where we can see there are only two 

In [493]:
file_exists = exists("data/credits_cleaned02.csv")
if(file_exists):
    credits = pd.read_csv('data/credits_cleaned02.csv',index_col=0)
    display(len(credits))
    display(list(credits.columns))

45399

['cast', 'id']

Each row contains a large json with information we don't wan't want to use, this is how each **cast** looks like

In [494]:
if(file_exists):
    display(credits.iloc[0]['cast'])

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

Since the format of the json is not compatible with **json.loads()** we need to change the structure in order to have a dict of people for each movie:
- Loop through each row on de dataset
- Change quotes to dummy values
- Remove unnecesary quotes or backslash (if any)
- Remove unnecesary indexes from the dict of people
- Update the dataset with the reduced json file

In [545]:
file_exists = exists("data/credits_cleaned02.csv")
if(file_exists):
    credits = pd.read_csv('data/credits_cleaned02.csv',index_col=0)
    credits.reset_index(drop=True, inplace=True)

errors = []
if(file_exists):
    for i, credit in credits.iterrows():
        if(i >= 0):
            try:
                cast = credit.cast
                cast = cast.replace("{'", "{##")
                cast = cast.replace('{"', "{##")

                cast = cast.replace("': ", "##: ")
                cast = cast.replace('": ', "##: ")

                cast = cast.replace(", '", ", ##")
                cast = cast.replace(', "', ", ##")

                cast = cast.replace("'}", "##}")
                cast = cast.replace('"}', "##}")

                cast = cast.replace(": '", ": ##")
                cast = cast.replace("', ", "##, ")
                cast = cast.replace(': "', ': ##')
                cast = cast.replace('", ', '##, ')
                cast = cast.replace('"}', '##}')
                cast = cast.replace("\\", '')

                cast = cast.replace('"', "")
                cast = cast.replace("'", "")

                cast = cast.replace('##', '"')
                cast = cast.replace(": None", ": null")
                if(credit.cast == "[]" or credit.cast == ""):
                    errors.append(i)
                else:
                    arrayOfPeople = json.loads(cast)
                    for dictOfPeople in arrayOfPeople:
                        dictOfPeople.pop("cast_id")
                        dictOfPeople.pop("credit_id")
                        dictOfPeople.pop("gender")
                        dictOfPeople.pop("order")
                        dictOfPeople.pop("profile_path")
                        dictOfPeople.pop("id")
                        dictOfPeople['character'].replace('"',"´")
                    credits.iloc[i,0] = json.dumps(arrayOfPeople)
            except Exception as e:
                #print(i)
                errors.append(i)


In [546]:
#These are out of range elements:
print(errors)

[137, 240, 393, 438, 595, 620, 661, 671, 679, 711, 741, 752, 756, 767, 771, 819, 833, 957, 1010, 1102, 1106, 1126, 1272, 1388, 1393, 1405, 1489, 1626, 1628, 1632, 1657, 1685, 1691, 1721, 1768, 1792, 2216, 2369, 2423, 2449, 2626, 2706, 2790, 2875, 2964, 3068, 3105, 3113, 3116, 3157, 3160, 3182, 3306, 3457, 3484, 3500, 3552, 3624, 3731, 3759, 3941, 3945, 4041, 4077, 4099, 4171, 4182, 4321, 4322, 4324, 4577, 4592, 4625, 4644, 4749, 4933, 5000, 5148, 5210, 5396, 5430, 5476, 5500, 5538, 5541, 5596, 5613, 5660, 5666, 5759, 5871, 5999, 6048, 6101, 6106, 6122, 6183, 6188, 6294, 6430, 6508, 6521, 6534, 6599, 6602, 6607, 6616, 6769, 6780, 6784, 6800, 6967, 6982, 7132, 7187, 7262, 7263, 7316, 7354, 7365, 7445, 7582, 7887, 7889, 7892, 7899, 7970, 8015, 8059, 8073, 8222, 8237, 8249, 8280, 8609, 8780, 8795, 8947, 9003, 9054, 9202, 9203, 9215, 9231, 9275, 9281, 9322, 9346, 9397, 9424, 9460, 9475, 9503, 9504, 9711, 9751, 9816, 9872, 9974, 9986, 10078, 10123, 10226, 10436, 10470, 10527, 10568, 10576, 1

In [547]:
len(credits)

45399

In [548]:
#If we want to eliminate errors:
for i in errors:
    try:
        credits.drop(credits.index[i],inplace=True)
    except Exception as e:
        print(e)
        pass
credits.reset_index(drop=True, inplace=True)

index 43185 is out of bounds for axis 0 with size 43182
index 43192 is out of bounds for axis 0 with size 43182
index 43211 is out of bounds for axis 0 with size 43182
index 43212 is out of bounds for axis 0 with size 43182
index 43240 is out of bounds for axis 0 with size 43182
index 43245 is out of bounds for axis 0 with size 43182
index 43246 is out of bounds for axis 0 with size 43182
index 43254 is out of bounds for axis 0 with size 43182
index 43264 is out of bounds for axis 0 with size 43182
index 43271 is out of bounds for axis 0 with size 43182
index 43281 is out of bounds for axis 0 with size 43182
index 43297 is out of bounds for axis 0 with size 43182
index 43317 is out of bounds for axis 0 with size 43182
index 43356 is out of bounds for axis 0 with size 43182
index 43361 is out of bounds for axis 0 with size 43182
index 43366 is out of bounds for axis 0 with size 43182
index 43367 is out of bounds for axis 0 with size 43182
index 43382 is out of bounds for axis 0 with siz

In [549]:
len(credits)

43182

In [550]:
credits.head(3)

,cast,id
0,"[{""character"": ""Woody (voice)"", ""name"": ""Tom H...",862
1,"[{""character"": ""Alan Parrish"", ""name"": ""Robin ...",8844
2,"[{""character"": ""Max Goldman"", ""name"": ""Walter ...",15602


Now we store this dataframe as a new file, with these steps we have reduced the original file from 190mb to 37mb which is better because we will deal with a smaller file

In [551]:
credits.to_csv('data/credits_cleaned03.csv')

## USING REDUCED CREDITS FILE 

In [552]:
credits = pd.read_csv('data/credits_cleaned03.csv', index_col=0)
credits.head(2)

,cast,id
0,"[{""character"": ""Woody (voice)"", ""name"": ""Tom H...",862
1,"[{""character"": ""Alan Parrish"", ""name"": ""Robin ...",8844


Let's get a dataset of performers counting their participation on movies:

In [554]:
actArray = []
movArray = []
for i, credit in credits.iterrows():
    try:
        credit.cast = credit.cast.replace("'", '"')
        arrayOfPeople = json.loads(credit.cast)
        for dictOfPeople in arrayOfPeople:
            actArray.append(dictOfPeople["name"])
            movArray.append(credit.id)
    except Exception as e:  
        print(i,credit.cast)
        break

    
myDict = {'act': actArray, 'mov': movArray}
acts = pd.DataFrame(data=myDict)

(acts['act'].value_counts())

Bess Flowers            230
Christopher Lee         141
Samuel L. Jackson       120
John Wayne              120
Donald Sutherland       107
                       ... 
Joanne                    1
Johnny Chung-Jen Lin      1
Ma Nien-Hsien             1
Wei-min Ying              1
Aleksandr Chabrov         1
Name: act, Length: 196194, dtype: int64